In [1]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
from feature.moving_average import moving_average
from feature.utils import tanspose_data
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [2]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [3]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)
train_rol = tanspose_data(train_input_list)
test_rol = tanspose_data(test_input_list)

In [4]:
train = train.reset_index()
test = test.reset_index()
train_rol = train_rol.reset_index()
test_rol = test_rol.reset_index()

In [5]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [6]:
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

In [7]:
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']
train = train.drop(drop_feature, axis=1)
test = test.drop(drop_feature, axis=1)


In [10]:
X = train.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
y = train['predicted_weight_g']
from sklearn.model_selection import StratifiedKFold, KFold

skf = KFold(n_splits=5, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    CAT = CatBoostRegressor(verbose=500, early_stopping_rounds=50, task_type="GPU", random_seed=113)
    CAT.fit(x_train, y_train)

    y_pred = CAT.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    CAT_model[f] = CAT
    print(f'================================================================================\n\n')


submit = test[['DAT', 'predicted_weight_g']]
x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time'], axis=1)
for fold in range(5):
    submit['predicted_weight_g'] += CAT_model[fold].predict(x_test)/5
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)

====================================1============================================
Learning rate set to 0.037997
0:	learn: 41.5997940	total: 10.6ms	remaining: 10.5s
500:	learn: 3.7348874	total: 4.93s	remaining: 4.91s
999:	learn: 2.8179735	total: 9.82s	remaining: 0us
1 Fold RMSE = 8.104316392783497


====================================2============================================
Learning rate set to 0.037997
0:	learn: 40.0035784	total: 11.9ms	remaining: 11.9s
500:	learn: 4.6729542	total: 5.01s	remaining: 4.99s
999:	learn: 4.2719355	total: 9.89s	remaining: 0us
2 Fold RMSE = 8.265293466211762


====================================3============================================
Learning rate set to 0.037997
0:	learn: 41.1883462	total: 10.7ms	remaining: 10.7s
500:	learn: 4.3277725	total: 5.16s	remaining: 5.14s
999:	learn: 3.4371248	total: 10.2s	remaining: 0us
3 Fold RMSE = 6.7506789467683905


====================================4============================================
Learning rate set

In [11]:
df_imp = pd.DataFrame({'imp':CAT.feature_importances_}, index = CAT.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('ec관측치', 20)",0.000062
"('일간누적분무량', 13)",0.000213
"('ec관측치', 13)",0.000227
"('일간누적분무량', 5)",0.001132
"('ec관측치', 16)",0.001279
...,...
"('내부온도관측치', 14)",2.566168
월간누적적색광량,11.759637
월간누적백색광량,11.800608
월간누적분무량,18.306652


In [12]:
df_imp.to_csv('zz.csv')